In [ ]:
import pandas as pd

# Load the dataset from a CSV file
def load_dataset(file_path):
    dataset = pd.read_csv(file_path)

    # For this example, assuming 'article' is the column containing the articles
    # Adjust this based on the actual column names in your dataset
    dataset = dataset.rename(columns={"article": "text"})

    return dataset

# Specify the file path for your dataset
file_path = "/content/train.csv"

# Load the dataset
dataset = load_dataset(file_path)

# Display the first few rows of the dataset
print(dataset.head())

ParserError: Error tokenizing data. C error: EOF inside string starting at row 217285

In [ ]:
import pandas as pd

def load_dataset(file_path):
    try:
        # Try loading the dataset
        dataset = pd.read_csv(file_path)
        return dataset
    except pd.errors.ParserError:
        # If there's a parsing error, handle it
        print("Parsing error encountered. Attempting to clean the data...")
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Clean the data (example: remove quotes)
        cleaned_lines = [line.replace('"', '') for line in lines]

        # Write cleaned data to a temporary file
        temp_file_path = 'temp.csv'
        with open(temp_file_path, 'w') as temp_file:
            temp_file.writelines(cleaned_lines)

        # Load the cleaned dataset
        dataset = pd.read_csv(temp_file_path)

        # Remove temporary file
        import os
        os.remove(temp_file_path)

        return dataset

# Example usage:
file_path = '/content/test.csv'
dataset = load_dataset(file_path)
print(dataset.head())


                                         id  \
0  92c514c913c0bdfe25341af9fd72b29db544099b   
1  2003841c7dc0e7c5b1a248f9cd536d727f27a45a   
2  91b7d2311527f5c2b63a65ca98d21d9c92485149   
3  caabf9cbdf96eb1410295a673e953d304391bfbb   
4  3da746a7d9afcaa659088c8366ef6347fe6b53ea   

                                             article  \
0  Ever noticed how plane seats appear to be gett...   
1  A drunk teenage boy had to be rescued by secur...   
2  Dougie Freedman is on the verge of agreeing a ...   
3  Liverpool target Neto is also wanted by PSG an...   
4  Bruce Jenner will break his silence in a two-h...   

                                          highlights  
0  Experts question if  packed out planes are put...  
1  Drunk teenage boy climbed into lion enclosure ...  
2  Nottingham Forest are close to extending Dougi...  
3  Fiorentina goalkeeper Neto has been linked wit...  
4  Tell-all interview with the reality TV star, 6...  


In [ ]:
import csv

In [ ]:
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Define the preprocess_text function
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    sentences = sent_tokenize(text)
    preprocessed_sentences = []
    for sent in sentences:
        words = word_tokenize(sent)
        words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]
        preprocessed_sentences.append(" ".join(words))
    return preprocessed_sentences

In [ ]:
# Apply TextRank algorithm for text summarization
def textrank_summary(text, num_sentences=3):
    preprocessed_sentences = preprocess_text(text)

    sentence_similarity = np.zeros((len(preprocessed_sentences), len(preprocessed_sentences)))
    for i in range(len(preprocessed_sentences)):
        for j in range(len(preprocessed_sentences)):
            if i != j:
                sentence1 = set(preprocessed_sentences[i].split())
                sentence2 = set(preprocessed_sentences[j].split())
                similarity_score = len(sentence1.intersection(sentence2)) / (np.log(len(sentence1)) + np.log(len(sentence2)) + 1e-6)
                sentence_similarity[i][j] = similarity_score

    scores = np.ones(len(preprocessed_sentences))
    damping_factor = 0.85
    for _ in range(100):
        new_scores = (1 - damping_factor) + damping_factor * np.sum(sentence_similarity, axis=1)
        if np.allclose(new_scores, scores, rtol=1e-4):
            break
        scores = new_scores

    top_sentences_idx = np.argsort(-scores)[:num_sentences]
    summary = [preprocessed_sentences[i] for i in top_sentences_idx]

    return " ".join(summary)


In [ ]:
# Apply TextRank summarization to the dataset
train['text_column'] = train['article'].apply(lambda x: textrank_summary(x, num_sentences=3))
train.to_csv('dataset_with_summaries_text_rank.csv', index=False)
print("Summaries saved to 'dataset_with_summaries_text_rank.csv'.")


NameError: name 'train' is not defined

In [ ]:
# Ensure 'article' column is string-like by converting to string
dataset['article'] = dataset['article'].astype(str)


In [ ]:
# Define the preprocess_text function
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    sentences = sent_tokenize(text)
    preprocessed_sentences = []
    for sent in sentences:
        words = word_tokenize(sent)
        words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]
        preprocessed_sentences.append(" ".join(words))
    return preprocessed_sentences

# Apply TextRank algorithm for text summarization
def textrank_summary(text, num_sentences=3):
    preprocessed_sentences = preprocess_text(text)

    sentence_similarity = np.zeros((len(preprocessed_sentences), len(preprocessed_sentences)))
    for i in range(len(preprocessed_sentences)):
        for j in range(len(preprocessed_sentences)):
            if i != j:
                sentence1 = set(preprocessed_sentences[i].split())
                sentence2 = set(preprocessed_sentences[j].split())
                similarity_score = len(sentence1.intersection(sentence2)) / (np.log(len(sentence1)) + np.log(len(sentence2)) + 1e-6)
                sentence_similarity[i][j] = similarity_score

    scores = np.ones(len(preprocessed_sentences))
    damping_factor = 0.85
    for _ in range(100):
        new_scores = (1 - damping_factor) + damping_factor * np.sum(sentence_similarity, axis=1)
        if np.allclose(new_scores, scores, rtol=1e-4):
            break
        scores = new_scores

    top_sentences_idx = np.argsort(-scores)[:num_sentences]
    summary = [preprocessed_sentences[i] for i in top_sentences_idx]

    return " ".join(summary)

In [ ]:
# Apply TextRank summarization to the dataset
dataset['text_column'] = dataset['article'].apply(lambda x: textrank_summary(x, num_sentences=3))
train.to_csv('dataset_with_summaries_text_rank.csv', index=False)
print("Summaries saved to 'dataset_with_summaries_text_rank.csv'.")


In [ ]:
# Apply TextRank summarization to the dataset
dataset['text_column'] = dataset['article'].apply(lambda x: textrank_summary(x, num_sentences=3))
dataset.to_csv('dataset_with_summaries_text_rank.csv', index=False)
print("Summaries saved to 'dataset_with_summaries_text_rank.csv'.")

In [ ]:
# Display the first few rows of the dataset with summaries
print(train.head())

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

nltk.download('stopwords')
nltk.download('punkt')

import pandas as pd

# Function to load the dataset with error handling
def load_dataset(file_path):
    try:
        dataset = pd.read_csv(file_path, error_bad_lines=False)  # Skip problematic rows
    except pd.errors.ParserError as e:
        print(f"Error while parsing CSV: {e}")
        # Handle the error as needed, e.g., skip the row or display an error message
        dataset = pd.DataFrame()  # Or provide an empty DataFrame

    return dataset

# Specify the file path for your dataset
file_path = "test.csv"

# Load the dataset with error handling
dataset = load_dataset(file_path)

if not dataset.empty:
    # Data processing steps go here
    print(dataset.head())

# Data Preprocessing
stop_words = set(stopwords.words('english'))

# Function to clean text
def clean_text(text):
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub('"', '', text)
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    return text

df_cnn['cleaned_article'] = df_cnn['article'].apply(clean_text)

# Tokenization
def tokenize_sentences(article):
    sentences = sent_tokenize(article)
    return sentences

# Function to generate article summary
def generate_summary(article, n=5):
    sentences = tokenize_sentences(article)
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)
    similarity_matrix = cosine_similarity(X, X)
    nx_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i], sentence) for i, sentence in enumerate(sentences)), reverse=True)
    summary_sentences = [sentence for _, sentence in ranked_sentences[:n]]
    return ' '.join(summary_sentences)

# Function to get the article and its summary based on ID
def get_summary_by_id(dataframe, id):
    row = dataframe[dataframe['id'] == id]
    if not row.empty:
        article = row['article'].values[0]
        summary = generate_summary(article)
        return article, summary
    else:
        return "ID not found in the dataset", ""

# Example usage
article_id = '0054d6d30dbcad772e20b22771153a2a9cbeaf62'
article, summary = get_summary_by_id(df_cnn, article_id)
print("Article:\n", article)
print("\nSummary:\n", summary)

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

nltk.download('stopwords')
nltk.download('punkt')

# Function to load the dataset with error handling
def load_dataset(file_path):
    try:
        dataset = pd.read_csv(file_path, error_bad_lines=False)  # Skip problematic rows
    except pd.errors.ParserError as e:
        print(f"Error while parsing CSV: {e}")
        # Handle the error as needed, e.g., skip the row or display an error message
        dataset = pd.DataFrame()  # Or provide an empty DataFrame

    return dataset

# Specify the file path for your dataset
file_path = "test.csv"

# Load the dataset with error handling
dataset = load_dataset(file_path)

if not dataset.empty:
    # Data preprocessing: Cleaning the article text
    stop_words = set(stopwords.words('english'))

    # Function to clean text
    def clean_text(text):
        text = re.sub(r'\([^)]*\)', '', text)
        text = re.sub('"', '', text)
        text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
        return text

    dataset['cleaned_article'] = dataset['article'].apply(clean_text)

    # Tokenization
    def tokenize_sentences(article):
        sentences = sent_tokenize(article)
        return sentences

    # Function to generate article summary
    def generate_summary(article, n=5):
        sentences = tokenize_sentences(article)
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(sentences)
        similarity_matrix = cosine_similarity(X, X)
        nx_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(nx_graph)
        ranked_sentences = sorted(((scores[i], sentence) for i, sentence in enumerate(sentences)), reverse=True)
        summary_sentences = [sentence for _, sentence in ranked_sentences[:n]]
        return ' '.join(summary_sentences)

    # Function to get the article and its summary based on ID
    def get_summary_by_id(dataframe, id):
        row = dataframe[dataframe['id'] == id]
        if not row.empty:
            article = row['article'].values[0]
            summary = generate_summary(article)
            return article, summary
        else:
            return "ID not found in the dataset", ""

    # Example usage
    article_id = '0054d6d30dbcad772e20b22771153a2a9cbeaf62'
    article, summary = get_summary_by_id(dataset, article_id)
    print("Article:\n", article)
    print("\nSummary:\n", summary)

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

nltk.download('stopwords')
nltk.download('punkt')

# Function to load the dataset with error handling
def load_dataset(file_path):
    try:
        dataset = pd.read_csv(file_path, error_bad_lines=False, skiprows=range(10863, None))  # Skip problematic rows
    except pd.errors.ParserError as e:
        print(f"Error while parsing CSV: {e}")
        # Handle the error as needed, e.g., skip the row or display an error message
        dataset = pd.DataFrame()  # Or provide an empty DataFrame

    return dataset

# Specify the file path for your dataset
file_path = "test.csv"

# Load the dataset with error handling
dataset = load_dataset(file_path)

if not dataset.empty:
    # Data preprocessing: Cleaning the article text
    stop_words = set(stopwords.words('english'))

    # Function to clean text
    def clean_text(text):
        text = re.sub(r'\([^)]*\)', '', text)
        text = re.sub('"', '', text)
        text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
        return text

    dataset['cleaned_article'] = dataset['article'].apply(clean_text)

    # Tokenization
    def tokenize_sentences(article):
        sentences = sent_tokenize(article)
        return sentences

    # Function to generate article summary
    def generate_summary(article, n=5):
        sentences = tokenize_sentences(article)
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(sentences)
        similarity_matrix = cosine_similarity(X, X)
        nx_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(nx_graph)
        ranked_sentences = sorted(((scores[i], sentence) for i, sentence in enumerate(sentences)), reverse=True)
        summary_sentences = [sentence for _, sentence in ranked_sentences[:n]]
        return ' '.join(summary_sentences)

    # Function to get the article and its summary based on ID
    def get_summary_by_id(dataframe, id):
        row = dataframe[dataframe['id'] == id]
        if not row.empty:
            article = row['article'].values[0]
            summary = generate_summary(article)
            return article, summary
        else:
            return "ID not found in the dataset", ""

    # Example usage
    article_id = '0054d6d30dbcad772e20b22771153a2a9cbeaf62'
    article, summary = get_summary_by_id(dataset, article_id)
    print("Article:\n", article)
    print("\nSummary:\n", summary)

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

nltk.download('stopwords')
nltk.download('punkt')

# Function to load the dataset with error handling
def load_dataset(file_path):
    try:
        dataset = pd.read_csv(file_path, error_bad_lines=False, skiprows=lambda x: x >= 10863)  # Skip rows starting from 10863
    except pd.errors.ParserError as e:
        print(f"Error while parsing CSV: {e}")
        # Handle the error as needed, e.g., skip the row or display an error message
        dataset = pd.DataFrame()  # Or provide an empty DataFrame

    return dataset

# Specify the file path for your dataset
file_path = "test.csv"

# Load the dataset with error handling
dataset = load_dataset(file_path)

if not dataset.empty:
    # Data preprocessing: Cleaning the article text
    stop_words = set(stopwords.words('english'))

    # Function to clean text
    def clean_text(text):
        text = re.sub(r'\([^)]*\)', '', text)
        text = re.sub('"', '', text)
        text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
        return text

    dataset['cleaned_article'] = dataset['article'].apply(clean_text)

    # Tokenization
    def tokenize_sentences(article):
        sentences = sent_tokenize(article)
        return sentences

    # Function to generate article summary
    def generate_summary(article, n=5):
        sentences = tokenize_sentences(article)
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(sentences)
        similarity_matrix = cosine_similarity(X, X)
        nx_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(nx_graph)
        ranked_sentences = sorted(((scores[i], sentence) for i, sentence in enumerate(sentences)), reverse=True)
        summary_sentences = [sentence for _, sentence in ranked_sentences[:n]]
        return ' '.join(summary_sentences)

    # Function to get the article and its summary based on ID
    def get_summary_by_id(dataframe, id):
        row = dataframe[dataframe['id'] == id]
        if not row.empty:
            article = row['article'].values[0]
            summary = generate_summary(article)
            return article, summary
        else:
            return "ID not found in the dataset", ""

    # Example usage
if not dataset.empty:
    article_id = '92c514c913c0bdfe25341af9fd72b29db544099b'
    article, summary = get_summary_by_id(dataset, article_id)

    # Counting words in the article and summary
    article_word_count = count_words(article)
    summary_word_count = count_words(summary)

    print("Article Word Count:", article_word_count)
    print("Summary Word Count:", summary_word_count)


In [ ]:
from nltk.tokenize import word_tokenize
import pandas as pd

In [ ]:
import pandas as pd
import networkx as nx
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import re
from string import punctuation
from heapq import nlargest

# Load the dataset
df = pd.read_csv("test.csv")

# Contractions dictionary and regex
contractions_dict = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y’all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you're": "you are",
    "you've": "you have",
}

contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function to clean the HTML from the article

In [ ]:
# Function to clean HTML tags from the article
def clean_html(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

# Function to expand contractions
def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

In [ ]:
# Preprocessing function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = clean_html(text)  # Remove HTML tags
    text = expand_contractions(text)  # Expand contractions
    text = re.sub(r'\S+@\S+', '', text)  # Remove email addresses
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)  # Remove URLs
    text = text.replace("\xa0", " ")  # Remove '\xa0'
    text = text.replace("'s", '')  # Strip possessives
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespaces
    return text


In [ ]:
# Function to preprocess the entire article
def preprocess_article(article):
    return preprocess_text(article)

# Function to preprocess the article and tokenize it into sentences
def preprocess_and_tokenize(article):
    sentences = sent_tokenize(preprocess_article(article))
    return sentences

In [ ]:
# Function to create a graph from sentences
def create_graph(sentences):
    graph = nx.Graph()
    graph.add_nodes_from(range(len(sentences)))
    vectorizer = CountVectorizer().fit_transform(sentences)
    similarity_matrix = cosine_similarity(vectorizer)
    for i in range(len(sentences)):
        for j in range(i+1, len(sentences)):
            graph.add_edge(i, j, weight=similarity_matrix[i][j])
    return graph

In [ ]:
# Function to rank sentences using TextRank algorithm
def text_rank(graph):
    scores = nx.pagerank(graph)
    return scores

# Function to generate summary
def generate_summary(article, scores, top_n=3):
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(article)), reverse=True)
    summary = ' '.join([sentence[1] for sentence in ranked_sentences[:top_n]])
    return summary

# Main function
def main(article_id):
    # Get article based on article_id
    article = df.loc[df['id'] == article_id]['article'].values[0]

    # Preprocess the article and tokenize into sentences
    preprocessed_sentences = preprocess_and_tokenize(article)

    # Create graph from sentences
    graph = create_graph(preprocessed_sentences)

    # Rank sentences using TextRank algorithm
    scores = text_rank(graph)

    # Generate summary
    summary = generate_summary(preprocessed_sentences, scores)

    # Return article and summary
    return article, summary

# Test the function
article_id = '92c514c913c0bdfe25341af9fd72b29db544099b'  # Example ID
article, summary = main(article_id)

# Count the number of words in article and summary
article_word_count = len(article.split())
summary_word_count = len(summary.split())

print("Article:\n", article)
print("\nSummary:\n", summary)
print("\nNumber of words in article:", article_word_count)
print("Number of words in summary:", summary_word_count)

Article:
 Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting 

In [ ]:
pip install rouge


In [ ]:
from rouge import Rouge

# Function to calculate ROUGE scores
def calculate_rouge(reference_summary, generated_summary):
    rouge = Rouge()
    scores = rouge.get_scores(generated_summary, reference_summary)
    return scores

# Reference summary (highlight) from the dataset
reference_summary = df.loc[df['id'] == article_id]['highlights'].values[0]

# Calculate ROUGE scores
rouge_scores = calculate_rouge(reference_summary, summary)

# Print ROUGE scores
print("ROUGE scores:")
print(rouge_scores)


ROUGE scores:
[{'rouge-1': {'r': 0.3939393939393939, 'p': 0.17567567567567569, 'f': 0.2429906499397328}, 'rouge-2': {'r': 0.09090909090909091, 'p': 0.031578947368421054, 'f': 0.04687499617309602}, 'rouge-l': {'r': 0.3939393939393939, 'p': 0.17567567567567569, 'f': 0.2429906499397328}}]


In [ ]:
import textwrap
from transformers import pipeline

In [ ]:
# Test the function
article_id = '92c514c913c0bdfe25341af9fd72b29db544099b'  # Example ID
article, summary = main(article_id)

# Count the number of words in article and summary
article_word_count = len(article.split())
summary_word_count = len(summary.split())

print("Article:\n", article)
print("\nSummary:\n", summary)
print("\nNumber of words in article:", article_word_count)
print("Number of words in summary:", summary_word_count)

# Word wrap article and summary
wrapper = textwrap.TextWrapper(width=80)
article_wrapped = wrapper.fill(text=article)
summary_wrapped = wrapper.fill(text=summary)

# Print article, summary, and word counts
print("Article:\n", article_wrapped)
print("\nSummary:\n", summary_wrapped)

from rouge import Rouge

# Function to calculate ROUGE scores
def calculate_rouge(reference_summary, generated_summary):
    rouge = Rouge()
    scores = rouge.get_scores(generated_summary, reference_summary)
    return scores

# Reference summary (highlight) from the dataset
reference_summary = df.loc[df['id'] == article_id]['highlights'].values[0]

# Calculate ROUGE scores
rouge_scores = calculate_rouge(reference_summary, summary)

# Print ROUGE scores
print("ROUGE scores:")
print(rouge_scores)


Article:
 Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting 

In [ ]:
from transformers import pipeline

# Load the BERT-based text summarization pipeline
summarization_pipeline = pipeline("summarization")

# Function to generate summary using BERT
def generate_summary_bert(article):
    summarized_text = summarization_pipeline(article, max_length=150, min_length=30, do_sample=False)
    summary = summarized_text[0]['summary_text']
    return summary

# Main function with BERT summarization
def main_bert(article_id):
    # Get article based on article_id
    article = df.loc[df['id'] == article_id]['article'].values[0]

    # Generate summary using BERT
    summary = generate_summary_bert(article)

    # Return article and summary
    return article, summary

# Test the BERT-based summarization function
article_id = '92c514c913c0bdfe25341af9fd72b29db544099b'  # Example ID
article, summary_bert = main_bert(article_id)

# Print the BERT-generated summary
print("\nBERT-generated Summary:\n", summary_bert)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]


BERT-generated Summary:
  U.S consumer advisory group set up by the Department of Transportation said that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans . Many economy seats on United Airlines have 30 inches of room, while some airlines offer as little as 28 inches .


In [ ]:
from transformers import pipeline

# Load the BERT-based text summarization pipeline
summarization_pipeline = pipeline("summarization")

# Function to generate summary using BERT
def generate_summary_bert(article):
    summarized_text = summarization_pipeline(article, max_length=150, min_length=30, do_sample=False)
    summary = summarized_text[0]['summary_text']
    return summary

# Main function with BERT summarization
def main_bert(article_id):
    # Get article based on article_id
    article = df.loc[df['id'] == article_id]['article'].values[0]

    # Generate summary using BERT
    summary = generate_summary_bert(article)

    # Count the number of words in article and summary
    article_word_count = len(article.split())
    summary_word_count = len(summary.split())

    # Print article, summary, and word counts
    print("Article:\n", article)
    print("\nSummary:\n", summary)
    print("\nNumber of words in article:", article_word_count)
    print("Number of words in summary:", summary_word_count)

# Test the BERT-based summarization function
article_id = '92c514c913c0bdfe25341af9fd72b29db544099b'  # Example ID
main_bert(article_id)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Article:
 Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting 

In [ ]:
import textwrap
from transformers import pipeline

# Load the BERT-based text summarization pipeline
summarization_pipeline = pipeline("summarization")

# Function to generate summary using BERT
def generate_summary_bert(article):
    summarized_text = summarization_pipeline(article, max_length=150, min_length=30, do_sample=False)
    summary = summarized_text[0]['summary_text']
    return summary

# Main function with BERT summarization
def main_bert(article_id):
    # Get article based on article_id
    article = df.loc[df['id'] == article_id]['article'].values[0]

    # Generate summary using BERT
    summary = generate_summary_bert(article)

    # Count the number of words in article and summary
    article_word_count = len(article.split())
    summary_word_count = len(summary.split())

    # Word wrap article and summary
    wrapper = textwrap.TextWrapper(width=80)
    article_wrapped = wrapper.fill(text=article)
    summary_wrapped = wrapper.fill(text=summary)

    # Print article, summary, and word counts
    print("Article:\n", article_wrapped)
    print("\nSummary:\n", summary_wrapped)
    print("\nNumber of words in article:", article_word_count)
    print("Number of words in summary:", summary_word_count)

# Test the BERT-based summarization function
article_id = '92c514c913c0bdfe25341af9fd72b29db544099b'  # Example ID
main_bert(article_id)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Article:
 Ever noticed how plane seats appear to be getting smaller and smaller? With
increasing numbers of people taking to the skies, some experts are questioning
if having such packed out planes is putting passengers at risk. They say that
the shrinking space on aeroplanes is not only uncomfortable - it's putting our
health and safety in danger. More than squabbling over the arm rest, shrinking
space on planes putting our health and safety in danger? This week, a U.S
consumer advisory group set up by the Department of Transportation said at a
public hearing that while the government is happy to set standards for animals
flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In
a world where animals have more rights to space and food than humans,' said
Charlie Leocha, consumer representative on the committee. 'It is time that the
DOT and FAA take a stand for humane treatment of passengers.' But could crowding
on planes lead to more serious issues than fighting 

In [ ]:
import textwrap
from transformers import pipeline
from rouge import Rouge

# Load the BERT-based text summarization pipeline
summarization_pipeline = pipeline("summarization")

# Function to generate summary using BERT
def generate_summary_bert(article):
    summarized_text = summarization_pipeline(article, max_length=150, min_length=30, do_sample=False)
    summary = summarized_text[0]['summary_text']
    return summary

# Main function with BERT summarization
def main_bert(article_id):
    # Get article based on article_id
    article = df.loc[df['id'] == article_id]['article'].values[0]

    # Generate summary using BERT
    summary = generate_summary_bert(article)

    # Count the number of words in article and summary
    article_word_count = len(article.split())
    summary_word_count = len(summary.split())

    # Word wrap article and summary
    wrapper = textwrap.TextWrapper(width=80)
    article_wrapped = wrapper.fill(text=article)
    summary_wrapped = wrapper.fill(text=summary)

    # Print article, summary, and word counts
    print("Article:\n", article_wrapped)
    print("\nSummary:\n", summary_wrapped)
    print("\nNumber of words in article:", article_word_count)
    print("Number of words in summary:", summary_word_count)

    # Calculate ROUGE scores
    rouge = Rouge()
    rouge_scores = rouge.get_scores(summary, df.loc[df['id'] == article_id]['highlights'].values[0])
    print("\nROUGE scores:")
    print(rouge_scores)

# Test the BERT-based summarization function
article_id = '92c514c913c0bdfe25341af9fd72b29db544099b'  # Example ID
main_bert(article_id)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Article:
 Ever noticed how plane seats appear to be getting smaller and smaller? With
increasing numbers of people taking to the skies, some experts are questioning
if having such packed out planes is putting passengers at risk. They say that
the shrinking space on aeroplanes is not only uncomfortable - it's putting our
health and safety in danger. More than squabbling over the arm rest, shrinking
space on planes putting our health and safety in danger? This week, a U.S
consumer advisory group set up by the Department of Transportation said at a
public hearing that while the government is happy to set standards for animals
flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In
a world where animals have more rights to space and food than humans,' said
Charlie Leocha, consumer representative on the committee. 'It is time that the
DOT and FAA take a stand for humane treatment of passengers.' But could crowding
on planes lead to more serious issues than fighting 

In [ ]:
import textwrap
from transformers import pipeline
from rouge import Rouge

# Load the BERT-based text summarization pipeline
summarization_pipeline = pipeline("summarization")

# Function to generate summary using BERT
def generate_summary_bert(article):
    # Adjust max_length and min_length parameters to control summary length
    summarized_text = summarization_pipeline(article, max_length=300, min_length=150, do_sample=False)
    summary = summarized_text[0]['summary_text']
    return summary

# Main function with BERT summarization
def main_bert(article_id):
    # Get article based on article_id
    article = df.loc[df['id'] == article_id]['article'].values[0]

    # Generate summary using BERT
    summary = generate_summary_bert(article)

    # Count the number of words in article and summary
    article_word_count = len(article.split())
    summary_word_count = len(summary.split())

    # Word wrap article and summary
    wrapper = textwrap.TextWrapper(width=80)
    article_wrapped = wrapper.fill(text=article)
    summary_wrapped = wrapper.fill(text=summary)

    # Print article, summary, and word counts
    print("Article:\n", article_wrapped)
    print("\nSummary:\n", summary_wrapped)
    print("\nNumber of words in article:", article_word_count)
    print("Number of words in summary:", summary_word_count)

    # Calculate ROUGE scores
    rouge = Rouge()
    rouge_scores = rouge.get_scores(summary, df.loc[df['id'] == article_id]['highlights'].values[0])
    print("\nROUGE scores:")
    print(rouge_scores)

# Test the BERT-based summarization function
article_id = '92c514c913c0bdfe25341af9fd72b29db544099b'  # Example ID
main_bert(article_id)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Article:
 Ever noticed how plane seats appear to be getting smaller and smaller? With
increasing numbers of people taking to the skies, some experts are questioning
if having such packed out planes is putting passengers at risk. They say that
the shrinking space on aeroplanes is not only uncomfortable - it's putting our
health and safety in danger. More than squabbling over the arm rest, shrinking
space on planes putting our health and safety in danger? This week, a U.S
consumer advisory group set up by the Department of Transportation said at a
public hearing that while the government is happy to set standards for animals
flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In
a world where animals have more rights to space and food than humans,' said
Charlie Leocha, consumer representative on the committee. 'It is time that the
DOT and FAA take a stand for humane treatment of passengers.' But could crowding
on planes lead to more serious issues than fighting 

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the dataset
file_path = "test.csv"
dataset = pd.read_csv(file_path)

# Load GPT-2 model and tokenizer
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')

# Function for text summarization using GPT-2 model
def generate_summary_gpt2(text):
    inputs = gpt2_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = gpt2_model.generate(inputs, max_length=450, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = gpt2_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Find article and summary based on ID '92c514c913c0bdfe25341af9fd72b29db544099b'
article_gpt2 = dataset.loc[dataset['id'] == '92c514c913c0bdfe25341af9fd72b29db544099b']['article'].values[0]
summary_gpt2 = generate_summary_gpt2(article_gpt2)

# Print the article and summary
print("GPT-2 Summary:")
print("\nArticle:\n", article_gpt2)
print("\nSummary:\n", summary_gpt2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT-2 Summary:

Article:
 Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issue

In [ ]:
import pandas as pd

# Load the dataset
file_path = "test.csv"
dataset = pd.read_csv(file_path)

# Display column names
print("Column names in the dataset:")
print(dataset.columns)


Column names in the dataset:
Index(['id', 'article', 'highlights'], dtype='object')


In [ ]:
import pandas as pd
import textwrap
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the dataset
file_path = "test.csv"
dataset = pd.read_csv(file_path)

# Load GPT-2 model and tokenizer
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')

# Function for text summarization using GPT-2 model
def generate_summary_gpt2(text):
    inputs = gpt2_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = gpt2_model.generate(inputs, max_length=450, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = gpt2_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Function to wrap text and count words
def wrap_and_count(text):
    wrapped_text = textwrap.fill(text, width=80)
    word_count = len(text.split())
    return wrapped_text, word_count

# Find article and summary based on ID '92c514c913c0bdfe25341af9fd72b29db544099b'
article_id = '92c514c913c0bdfe25341af9fd72b29db544099b'
article_gpt2 = dataset.loc[dataset['id'] == article_id]['article'].values[0]
summary_gpt2 = generate_summary_gpt2(article_gpt2)

# Wrap article and summary and count words
wrapped_article, article_word_count = wrap_and_count(article_gpt2)
wrapped_summary, summary_word_count = wrap_and_count(summary_gpt2)

# Print the article, summary, and word count
print("GPT-2 Summary:")
print("\nArticle:\n", wrapped_article)
print("\nSummary:\n", wrapped_summary)
print("\nNumber of words in article:", article_word_count)
print("Number of words in summary:", summary_word_count)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT-2 Summary:

Article:
 Ever noticed how plane seats appear to be getting smaller and smaller? With
increasing numbers of people taking to the skies, some experts are questioning
if having such packed out planes is putting passengers at risk. They say that
the shrinking space on aeroplanes is not only uncomfortable - it's putting our
health and safety in danger. More than squabbling over the arm rest, shrinking
space on planes putting our health and safety in danger? This week, a U.S
consumer advisory group set up by the Department of Transportation said at a
public hearing that while the government is happy to set standards for animals
flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In
a world where animals have more rights to space and food than humans,' said
Charlie Leocha, consumer representative on the committee. 'It is time that the
DOT and FAA take a stand for humane treatment of passengers.' But could crowding
on planes lead to more serious issue

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from rouge import Rouge

# Load the dataset
file_path = "test.csv"
dataset = pd.read_csv(file_path)

# Load GPT-2 model and tokenizer
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')

# Function for text summarization using GPT-2 model
def generate_summary_gpt2(text):
    inputs = gpt2_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = gpt2_model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = gpt2_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Main function to process each article in the dataset
def process_articles(dataset):
    for index, row in dataset.iterrows():
        article = row['article']
        summary = generate_summary_gpt2(article)
        print("GPT-2 Summary:\n", summary)

# Call the main function
process_articles(dataset)


In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from rouge import Rouge

# Load the dataset
file_path = "test.csv"
dataset = pd.read_csv(file_path)

# Load GPT-2 model and tokenizer
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')

# Function for text summarization using GPT-2 model
def generate_summary_gpt2(text):
    inputs = gpt2_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = gpt2_model.generate(inputs, max_length=1000, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = gpt2_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Main function to process each article in the dataset
def process_articles(dataset):
    for index, row in dataset.iterrows():
        article = row['article']
        summary = generate_summary_gpt2(article)
        print("GPT-2 Summary:\n", summary)

# Call the main function
process_articles(dataset)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT-2 Summary:
 summarize: Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issu

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT-2 Summary:
 summarize: A drunk teenage boy had to be rescued by security after jumping into a lions' enclosure at a zoo in western India. Rahul Kumar, 17, clambered over the enclosure fence at the Kamla Nehru Zoological Park in Ahmedabad, and began running towards the animals, shouting he would 'kill them'. Mr Kumar explained afterwards that he was drunk and 'thought I'd stand a good chance' against the predators. Next level drunk: Intoxicated Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in Ahmedabad and began running towards the animals shouting 'Today I kill a lion!' Mr Kumar had been sitting near the enclosure when he suddenly made a dash for the lions, surprising zoo security. The intoxicated teenager ran towards the lions, shouting: 'Today I kill a lion or a lion kills me!' A zoo spokesman said: 'Guards had earlier spotted him close to the enclosure but had no idea he was planing to enter it. 'Fortunately, there are eight moats to cross before getting to where t

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT-2 Summary:
 summarize: The amount of time people spend listening to BBC radio has dropped to its lowest level ever, the corporation’s boss has admitted. Figures show that while millions still tune in, they listen for much shorter bursts. The average listener spent just ten hours a week tuning in to BBC radio in the last three months of 2014, according to official figures. The length of time people spend listening to BBC radio has dropped to its lowest level ever, figures show. This was 14 per cent down on a decade earlier, when listeners clocked up an average of 11.6 hours a week. The minutes of the BBC Trust’s February meeting, published yesterday, revealed that director general Tony Hall highlighted the fall. ‘He noted…that time spent listening to BBC radio had dropped to its lowest ever level,’ the documents said. Sources blamed the downward trend on people leading faster-paced lives than in the past, and a change in habits amongst young people. Lord Tony Hall, BBC director gene

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT-2 Summary:
 summarize: (CNN)So, you'd like a "Full House" reunion and spinoff? You got it, dude! Co-star John Stamos announced Monday night on "Jimmy Kimmel Live" that Netflix has ordered up a reunion special, followed by a spinoff series called "Fuller House." The show will feature Candace Cameron Bure, who played eldest daughter D.J. Tanner in the original series -- which aired from 1987 to 1995 -- as the recently widowed mother of three boys. "It's sort of a role reversal, and we turn the house over to her," Stamos told Kimmel. Jodie Sweetin, who played Stephanie Tanner in the original series, and Andrea Barber, who portrayed D.J.'s best friend Kimmy Gibbler, will both return for the new series, Netflix said. Stamos will produce and guest star. Talks with co-starsBob Saget, Mary-Kate and Ashley Olsen, Dave Coulier and Lori Loughlin are ongoing, Netflix said. The show will be available next year, Netflix said. "As big fans of the original Full House, we are thrilled to be able to

ValueError: Input length of input_ids is 1024, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.